In [ ]:
import pandas as pd
import re
import numpy as np
try:
    user_input = input("Enter a class file to grade (i.e. class1 for class1.txt):")
    with open(user_input.lower() + ".txt", "r") as file:
        pass
        # lưu trữ các data hợp lệ
        valid_data = []
        # lưu trữ các data lỗi mã SV
        N_invalid = []
        values_invalid = []
        # Lưu trữ các câu trả lời sai và count số lần trả lời sai
        wrong_ans = {}

        # Kiểm tra tính hợp lệ của dữ liệu 2
        for answer in file:
            answer_list = answer.split(",")
            if re.match(r"^N[0-9]{8}", answer_list[0]):
                if len(answer_list) == 26:
                    valid_data.append(answer)
                else:
                    values_invalid.append(answer)
            else:
                N_invalid.append(answer)

        # Tạo một list các câu hỏi hợp lệ để import vào Dataframe
        split_valid_data = []
        for data in valid_data:
            split_valid_data.append(data.split(","))
        df_valid_data = pd.DataFrame(split_valid_data)
        
        # Tạo một cột để tính tổng điểm của học sinh
        df_valid_data["Score"] = 0

        # Tính điểm
        answer_key = ("B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D").split(",")
        for index in range(0, len(df_valid_data.index)):
            for i in range(1, 26):
                if df_valid_data.loc[index, i].rstrip() == answer_key[i-1]:
                    df_valid_data.loc[index, "Score"] += 4
                elif df_valid_data.loc[index, i].rstrip() == "":
                    df_valid_data.loc[index, "Score"] += 0
                else:
                    df_valid_data.loc[index, "Score"] -= 1
                    if i not in wrong_ans:
                        wrong_ans[i] = 1
                    else:
                        wrong_ans[i] += 1

        #Tao một file lưu trữ Mã + điểm của học sinh
        file_path = str(user_input) + "_grades.txt"
        column = [0, "Score"]
        df_valid_data[column].to_csv(file_path, index = False, header = False)
        
except FileNotFoundError:
    print("File cannot be found")
else:
    print("Successfully opened file: " + file.name + "\n**** ANALYZING ****")

    # Task 2
    if len(N_invalid) == 0 and len(values_invalid) == 0:
        print("No errors found!")
    else:
        print("Invalid line of data N# is invalid:\n")
        for data in N_invalid:
            print(data)
        print("Invalid line of data does not contain exactly 26 values:\n")
        for data in values_invalid:
            print(data)
    print("**** REPORT ****\n" 
          + "Total valid lines of data:" 
          + str(len(valid_data)) 
          + "\nTotal invalid lines of data:" 
          + str(len(N_invalid) + len(values_invalid)))

    # Tạo báo cáo thống kê kết quả
    count_max_score = (df_valid_data['Score'] >= 80).sum()
    print("Total student of high scores:", count_max_score)
    print("Mean (average) score:", round(df_valid_data["Score"].mean(), 2))
    print("Highest score:", df_valid_data["Score"].max())
    print("Lowest score:", df_valid_data["Score"].min())
    print("Range of scores:", df_valid_data["Score"].max() - df_valid_data["Score"].min())
    print("Median score:", df_valid_data["Score"].median())
    
    # Thay thế giá trị trong bằng NaN
    skip_count = df_valid_data.replace("", np.nan, regex = True).isnull().sum()
    
    skip_count_max = skip_count[skip_count == skip_count.max()]
    print("Question that most people skip: ", end= "")
    for index in skip_count_max.index:
        count = skip_count_max[index]
        percentage = round(count * 1.0 / len(valid_data), 2)
        if index == skip_count_max.index.max():
            print(f"{index} - {count} - {percentage}", end = "\n")
        else:
            print(f"{index} - {count} - {percentage}", end = ", ")

    max_wrong = []
    for item in wrong_ans.items():
        if item[1] == max(wrong_ans.values()):
            max_wrong.append(item)
    print("Question that most people answer incorrectly: ", end = "")
    for i in range(len(max_wrong)):
        wrong_ans = max_wrong[i][0]
        count = max_wrong[i][1]
        pct = round(max_wrong[i][1] * 1.0 / len(valid_data), 2)
        if i == len(max_wrong) - 1:
            print(f"{wrong_ans} - {count} - {pct}", end = " ")
        else:
            print(f"{wrong_ans} - {count} - {pct}", end = ", ")